In [ ]:
# Dependencies
import json
import requests
import time
import pandas as pd
from config import uname, pword

In [ ]:
# Request credentials using username and password
url = "https://api.hooktheory.com/v1/users/auth"
header = {"accept": "application/json", 
          "Content-Type": "application/json",
          "username": uname,
          "password": pword}

response = requests.post(url, data=header).json()

#save the key and id from response
key = response['activkey']
user_id = response['id']

#create header used for queries
bearer = f'Bearer {key}'
a_header = {'Authorization':bearer}

In [ ]:
c = requests.session()
c.headers = a_header

In [ ]:
#set query url and parameters
url = "https://api.hooktheory.com/v1/trends/songs"
params = {'cp':'', 'page':1}

#list progressions we want to check
progressions = ['1,5,4,6', '1,4,5', '1,4,6,5', '6,4,1,5', '1,6,4,5']

#initialize a list to store data
songs = []

In [ ]:
comm = '''params['cp'] = progressions[0]
r = c.get(url, params = params)
response = r.json()
r.headers['X-Rate-Limit-Reset']'''

In [ ]:
for cp in progressions:
    params['cp'] = cp
    page = 1
    while page > 0:
        params['page'] = page
        r = c.get(url, params = params)
        count = int(r.headers['X-Rate-Limit-Remaining'])
        if count == 0:
            wait = int(r.headers['X-Rate-Limit-Reset'])
            print(len(songs))
            time.sleep(wait)
        response = r.json()
        if len(response) == 0:
            page = 0
        else:
            for song in response:
                try:
                    song['cp'] = cp
                    songs.append(song)
                except:
                    pass
            page += 1

In [ ]:
songs_df = pd.DataFrame(songs)
songs_df = songs_df.drop(columns='url')
songs_df.head(30)

In [ ]:
songs_df.to_csv('hooktheory_output.csv', index=False, header=True)

In [ ]:
songs_df.tail()